<a href="https://colab.research.google.com/github/Anirudh-SK2808/PERSONA/blob/main/Personality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install Required Libraries
!pip install librosa numpy tqdm --quiet

# ✅ Imports
import os
import librosa
import numpy as np
from tqdm import tqdm

# ✅ Paths (change if needed)
base_path = "/content/drive/MyDrive/datasets"
ravdess_path = os.path.join(base_path, "Audio_Speech_Actors_01-24")
savee_path = os.path.join(base_path, "ALL")
tess_path = os.path.join(base_path, "dataverse_files")

# ✅ Trait mapping from emotions
label_map = {
    'angry':     ['High', 'High', 'Clear', 'Assertive Leader'],
    'disgust':   ['Low', 'Low', 'Slurred', 'Critical Analyzer'],
    'fear':      ['Low', 'Medium', 'Rushed', 'Cautious Planner'],
    'happy':     ['High', 'High', 'Clear', 'Energetic Doer'],
    'sad':       ['Low', 'Low', 'Slurred', 'Reflective Introvert'],
    'calm':      ['Medium', 'Low', 'Clear', 'Calm Strategist'],
    'neutral':   ['Medium', 'Medium', 'Clear', 'Balanced Thinker'],
    'surprise':  ['High', 'High', 'Rushed', 'Expressive Idealist']
}

# ✅ Feature extractor
def extract_features(file_path, max_pad_len=174):
    try:
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
        delta = librosa.feature.delta(mfcc)
        def pad(x): return np.pad(x, ((0, 0), (0, max(0, max_pad_len - x.shape[1]))), mode='constant')[:, :max_pad_len]
        return np.vstack([pad(mfcc), pad(chroma), pad(contrast), pad(delta)])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# ✅ Emotion extractors
def extract_emotion_ravdess(filename):
    return {
        '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
        '05': 'angry', '06': 'fear', '07': 'disgust', '08': 'surprise'
    }.get(filename.split("-")[2])

def extract_emotion_savee(filename):
    return {
        'an': 'angry', 'di': 'disgust', 'fe': 'fear',
        'ha': 'happy', 'sa': 'sad', 'su': 'surprise', 'ne': 'neutral'
    }.get(filename[5:7].lower())

def extract_emotion_tess(filename):
    for key in label_map:
        if key in filename.lower():
            return key
    return None

# ✅ Load datasets
X, y_conf, y_energy, y_clarity, y_persona = [], [], [], [], []

def load_dataset(path, extractor, check_in_name=False):
    for root, _, files in os.walk(path):
        for file in tqdm(files, desc=f"Processing {os.path.basename(path)}"):
            if file.endswith(".wav"):
                emotion = extractor(file if check_in_name else os.path.basename(file))
                if emotion in label_map:
                    full_path = os.path.join(root, file)
                    features = extract_features(full_path)
                    if features is not None:
                        traits = label_map[emotion]
                        X.append(features)
                        y_conf.append(traits[0])
                        y_energy.append(traits[1])
                        y_clarity.append(traits[2])
                        y_persona.append(traits[3])

# ✅ Run extraction
load_dataset(ravdess_path, extract_emotion_ravdess)
load_dataset(savee_path, extract_emotion_savee)
load_dataset(tess_path, extract_emotion_tess, check_in_name=True)

# ✅ Save extracted data
np.save("X_features.npy", np.array(X))
np.save("y_conf.npy", np.array(y_conf))
np.save("y_energy.npy", np.array(y_energy))
np.save("y_clarity.npy", np.array(y_clarity))
np.save("y_persona.npy", np.array(y_persona))

print(f"✅ Saved {len(X)} samples.")

Processing Audio_Speech_Actors_01-24: 0it [00:00, ?it/s]
Processing dataverse_files: 100%|██████████| 11/11 [00:00<00:00, 31.72it/s]


✅ Saved 1449 samples.


In [ ]:
# ✅ Install required packages
!pip install matplotlib scikit-learn tensorflow --quiet

# ✅ Imports
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ✅ Load features and labels
X = np.load("X_features.npy")
y_conf = np.load("y_conf.npy")
y_energy = np.load("y_energy.npy")
y_clarity = np.load("y_clarity.npy")
y_persona = np.load("y_persona.npy")

# ✅ Reshape X and encode labels
X = X[..., np.newaxis]  # Add channel dimension
le_conf = LabelEncoder(); y_conf = le_conf.fit_transform(y_conf)
le_energy = LabelEncoder(); y_energy = le_energy.fit_transform(y_energy)
le_clarity = LabelEncoder(); y_clarity = le_clarity.fit_transform(y_clarity)
le_persona = LabelEncoder(); y_persona = le_persona.fit_transform(y_persona)

# ✅ Train/test split
y_all = list(zip(y_conf, y_energy, y_clarity, y_persona))
X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.2, stratify=y_conf, random_state=42)
y_train_split = list(zip(*y_train))
y_test_split = list(zip(*y_test))

# ✅ Build CNN model
input_layer = Input(shape=X.shape[1:])
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)

# Outputs
out_conf = Dense(len(le_conf.classes_), activation='softmax', name='confidence')(x)
out_energy = Dense(len(le_energy.classes_), activation='softmax', name='energy')(x)
out_clarity = Dense(len(le_clarity.classes_), activation='softmax', name='clarity')(x)
out_persona = Dense(len(le_persona.classes_), activation='softmax', name='persona')(x)

model = Model(inputs=input_layer, outputs=[out_conf, out_energy, out_clarity, out_persona])
model.compile(
    optimizer='adam',
        loss='sparse_categorical_crossentropy',
            metrics={
                    'confidence': 'accuracy',
                            'energy': 'accuracy',
                                    'clarity': 'accuracy',
                                            'persona': 'accuracy'
                                                }
                                                )
                                                model.summary()

                                                # ✅ Train model
                                                callbacks = [
                                                    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                                                        ModelCheckpoint("persona_model_best.h5", save_best_only=True)
                                                        ]

                                                        history = model.fit(
                                                            X_train, y_train_split,
                                                                validation_data=(X_test, y_test_split),
                                                                    epochs=50,
                                                                        batch_size=32,
                                                                            callbacks=callbacks
                                                                            )

                                                                            # ✅ Save model and encoders
                                                                            model.save("persona_model_final.h5")
                                                                            with open("encoders.pkl", "wb") as f:
                                                                                pickle.dump((le_conf, le_energy, le_clarity, le_persona), f)

                                                                                # ✅ Evaluate
                                                                                loss, acc_conf, acc_energy, acc_clarity, acc_persona = model.evaluate(X_test, y_test_split)
                                                                                print("\n🌟 Test Accuracies:")
                                                                                print(f"   Confidence: {acc_conf:.2f}")
                                                                                print(f"   Energy:     {acc_energy:.2f}")
                                                                                print(f"   Clarity:    {acc_clarity:.2f}")
                                                                                print(f"   Persona:    {acc_persona:.2f}")

                                                                                # ✅ Plot confidence accuracy
                                                                                plt.plot(history.history['confidence_accuracy'], label='Train')
                                                                                plt.plot(history.history['val_confidence_accuracy'], label='Val')
                                                                                plt.title("Confidence Accuracy")
                                                                                plt.xlabel("Epochs")
                                                                                plt.ylabel("Accuracy")
                                                                                plt.legend()
                                                                                plt.grid()
                                                                                plt.show()


IndentationError: unexpected indent (ipython-input-6-329615692.py, line 63)

In [ ]:
# ✅ Install required packages (only needed in Colab)
!pip install matplotlib scikit-learn tensorflow --quiet

# ✅ Imports
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ✅ Load features and labels
X = np.load("X_features.npy")
y_conf = np.load("y_conf.npy")
y_energy = np.load("y_energy.npy")
y_clarity = np.load("y_clarity.npy")
y_persona = np.load("y_persona.npy")

# ✅ Reshape X and encode labels
X = X[..., np.newaxis]  # (N, 99, 174, 1)
le_conf = LabelEncoder()
y_conf = le_conf.fit_transform(y_conf)
le_energy = LabelEncoder()
y_energy = le_energy.fit_transform(y_energy)
le_clarity = LabelEncoder()
y_clarity = le_clarity.fit_transform(y_clarity)
le_persona = LabelEncoder()
y_persona = le_persona.fit_transform(y_persona)

# ✅ Split dataset
y_all = list(zip(y_conf, y_energy, y_clarity, y_persona))
X_train, X_test, y_train, y_test = train_test_split(
    X, y_all, test_size=0.2, stratify=y_conf, random_state=42
)

# ✅ Unzip for multi-output model
y_train_split = tuple(np.array(t) for t in zip(*y_train))
y_test_split = tuple(np.array(t) for t in zip(*y_test))

# ✅ Build model
input_layer = Input(shape=X.shape[1:])
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)

out_conf = Dense(len(le_conf.classes_), activation='softmax', name='confidence')(x)
out_energy = Dense(len(le_energy.classes_), activation='softmax', name='energy')(x)
out_clarity = Dense(len(le_clarity.classes_), activation='softmax', name='clarity')(x)
out_persona = Dense(len(le_persona.classes_), activation='softmax', name='persona')(x)

model = Model(inputs=input_layer, outputs=[out_conf, out_energy, out_clarity, out_persona])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', 'accuracy', 'accuracy', 'accuracy'])
model.summary()

# ✅ Train
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("persona_model_best.h5", save_best_only=True)
]

history = model.fit(
    X_train, y_train_split,
    validation_data=(X_test, y_test_split),
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)

# ✅ Save model and encoders
model.save("persona_model_final.h5")
with open("encoders.pkl", "wb") as f:
    pickle.dump((le_conf, le_energy, le_clarity, le_persona), f)

# ✅ Evaluate
loss, acc_conf, acc_energy, acc_clarity, acc_persona = model.evaluate(X_test, y_test_split)
print("\n🎯 Test Accuracies:")
print(f"   Confidence: {acc_conf:.2f}")
print(f"   Energy:     {acc_energy:.2f}")
print(f"   Clarity:    {acc_clarity:.2f}")
print(f"   Persona:    {acc_persona:.2f}")

# ✅ Plot accuracy of one output (confidence)
plt.plot(history.history['confidence_accuracy'], label='Train')
plt.plot(history.history['val_confidence_accuracy'], label='Val')
plt.title("Confidence Accuracy Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 99, 174,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 99, 174,   │        640 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 99, 174,   │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 49, 87,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 49, 87,    │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 49, 87,    │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 24, 43,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 132096)    │          0 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │ 33,816,832 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ confidence (Dense)  │ (None, 3)         │        771 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ energy (Dense)      │ (None, 3)         │        771 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ clarity (Dense)     │ (None, 3)         │        771 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ persona (Dense)     │ (None, 8)         │      2,056 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 33,896,465 (129.30 MB)

 Trainable params: 33,896,081 (129.30 MB)

 Non-trainable params: 384 (1.50 KB)

...

37/37 ━━━━━━━━━━━━━━━━━━━━ 192s 5s/step - clarity_accuracy: 0.4704 - clarity_loss: 1.0585 - confidence_accuracy: 0.3773 - confidence_loss: 1.0573 - energy_accuracy: 0.3923 - energy_loss: 1.0457 - loss: 5.2235 - persona_accuracy: 0.1282 - persona_loss: 2.0624 - val_clarity_accuracy: 0.4621 - val_clarity_loss: 1.0613 - val_confidence_accuracy: 0.4000 - val_confidence_loss: 1.0734 - val_energy_accuracy: 0.4000 - val_energy_loss: 1.0450 - val_loss: 5.2433 - val_persona_accuracy: 0.1207 - val_persona_loss: 2.0631
Epoch 34/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 162s 4s/step - clarity_accuracy: 0.4639 - clarity_loss: 1.0622 - confidence_accuracy: 0.4008 - confidence_loss: 1.0557 - energy_accuracy: 0.4009 - energy_loss: 1.0618 - loss: 5.2387 - persona_accuracy: 0.1372 - persona_loss: 2.0591 - val_clarity_accuracy: 0.4621 - val_clarity_loss: 1.0614 - val_confidence_accuracy: 0.4000 - val_confidence_loss: 1.0733 - val_energy_accuracy: 0.3966 - val_energy_loss: 1.0449 - val_loss: 5.2434 - val_persona_accu

37/37 ━━━━━━━━━━━━━━━━━━━━ 209s 5s/step - clarity_accuracy: 0.4493 - clarity_loss: 1.0705 - confidence_accuracy: 0.4260 - confidence_loss: 1.0469 - energy_accuracy: 0.4320 - energy_loss: 1.0430 - loss: 5.2132 - persona_accuracy: 0.1381 - persona_loss: 2.0531 - val_clarity_accuracy: 0.4621 - val_clarity_loss: 1.0612 - val_confidence_accuracy: 0.4000 - val_confidence_loss: 1.0736 - val_energy_accuracy: 0.3966 - val_energy_loss: 1.0444 - val_loss: 5.2432 - val_persona_accuracy: 0.1207 - val_persona_loss: 2.0629
Epoch 39/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - clarity_accuracy: 0.4580 - clarity_loss: 1.0654 - confidence_accuracy: 0.4147 - confidence_loss: 1.0511 - energy_accuracy: 0.3879 - energy_loss: 1.0680 - loss: 5.2504 - persona_accuracy: 0.1301 - persona_loss: 2.0658

37/37 ━━━━━━━━━━━━━━━━━━━━ 195s 4s/step - clarity_accuracy: 0.4583 - clarity_loss: 1.0651 - confidence_accuracy: 0.4143 - confidence_loss: 1.0513 - energy_accuracy: 0.3878 - energy_loss: 1.0676 - loss: 5.2498 - persona_accuracy: 0.1302 - persona_loss: 2.0657 - val_clarity_accuracy: 0.4621 - val_clarity_loss: 1.0614 - val_confidence_accuracy: 0.4000 - val_confidence_loss: 1.0738 - val_energy_accuracy: 0.3966 - val_energy_loss: 1.0446 - val_loss: 5.2432 - val_persona_accuracy: 0.1207 - val_persona_loss: 2.0629
Epoch 40/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 194s 4s/step - clarity_accuracy: 0.4660 - clarity_loss: 1.0608 - confidence_accuracy: 0.4133 - confidence_loss: 1.0523 - energy_accuracy: 0.4124 - energy_loss: 1.0530 - loss: 5.2229 - persona_accuracy: 0.1362 - persona_loss: 2.0565 - val_clarity_accuracy: 0.4621 - val_clarity_loss: 1.0612 - val_confidence_accuracy: 0.4000 - val_confidence_loss: 1.0736 - val_energy_accuracy: 0.3966 - val_energy_loss: 1.0442 - val_loss: 5.2432 - val_persona_accu

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 598ms/step - clarity_accuracy: 0.4891 - clarity_loss: 1.0477 - confidence_accuracy: 0.3779 - confidence_loss: 1.0606 - energy_accuracy: 0.3630 - energy_loss: 1.0647 - loss: 5.2460 - persona_accuracy: 0.1478 - persona_loss: 2.0729


ValueError: too many values to unpack (expected 5)

In [ ]:
# ✅ STEP 1: Install Required Libraries (only if not done)
!pip install librosa numpy tensorflow requests --quiet

# ✅ STEP 2: Imports
import numpy as np
import librosa
import requests
import tempfile
import os
import pickle
from tensorflow.keras.models import load_model

# ✅ STEP 3: Feature Extractor
def extract_features(file_path, max_pad_len=174):
    try:
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
        delta = librosa.feature.delta(mfcc)

        def pad(x): return np.pad(
            x, ((0, 0), (0, max(0, max_pad_len - x.shape[1]))),
            mode='constant'
        )[:, :max_pad_len]

        combined = np.vstack([pad(mfcc), pad(chroma), pad(contrast), pad(delta)])
        return combined
    except Exception as e:
        print("❌ Feature extraction failed:", e)
        return None

# ✅ STEP 4: Prediction Function
def predict_persona_from_url(url, model, le_conf, le_energy, le_clarity, le_persona):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print("❌ Failed to download file.")
            return

        # Save audio to temp file
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        # Extract features
        features = extract_features(tmp_path)
        if features is None:
            print("❌ Could not extract features.")
            return

        # Preprocess input
        X_input = features[np.newaxis, ..., np.newaxis]

        # Predict
        preds = model.predict(X_input)
        conf = le_conf.inverse_transform([np.argmax(preds[0])])[0]
        energy = le_energy.inverse_transform([np.argmax(preds[1])])[0]
        clarity = le_clarity.inverse_transform([np.argmax(preds[2])])[0]
        persona = le_persona.inverse_transform([np.argmax(preds[3])])[0]

        print("\n🔍 Prediction:")
        print(f"🗣️  Confidence: {conf}")
        print(f"⚡ Energy:     {energy}")
        print(f"🔊 Clarity:    {clarity}")
        print(f"🎭 Persona:    {persona}")

        os.remove(tmp_path)

    except Exception as e:
        print("❌ Error during prediction:", e)

# ✅ STEP 5: Load Model & Encoders
model = load_model("persona_model_final.h5")
with open("encoders.pkl", "rb") as f:
    le_conf, le_energy, le_clarity, le_persona = pickle.load(f)

# ✅ STEP 6: Provide Test URL
test_url = "https://github.com/harvitronix/five-minutes-a-day-ml/raw/master/datasets/audio/03-01-05-01-01-01-01.wav"

# ✅ STEP 7: Run Prediction
predict_persona_from_url(
    url=test_url,
    model=model,
    le_conf=le_conf,
    le_energy=le_energy,
    le_clarity=le_clarity,
    le_persona=le_persona
)

❌ Failed to download file.


In [ ]:
# Evaluate and print each accuracy metric
results = model.evaluate(X_test, y_test_split, verbose=0)

print("\n🎯 Test Accuracies:")
print(f"   Confidence: {results[5]:.2f}")
print(f"   Energy:     {results[6]:.2f}")
print(f"   Clarity:    {results[7]:.2f}")
print(f"   Persona:    {results[8]:.2f}")



🎯 Test Accuracies:
   Confidence: 0.46
   Energy:     0.40
   Clarity:    0.40
   Persona:    0.12


In [ ]:
# ✅ Install requirements (if needed in Colab)
!pip install librosa --quiet

# ✅ Imports
import librosa
import numpy as np
import pickle
from tensorflow.keras.models import load_model

# ✅ Load model and encoders
model = load_model("persona_model_final.h5")
with open("encoders.pkl", "rb") as f:
    le_conf, le_energy, le_clarity, le_persona = pickle.load(f)

# ✅ Prediction function for local .wav file
def predict_persona_from_local_file(file_path, model, le_conf, le_energy, le_clarity, le_persona):
    try:
        # Step 1: Load and extract features
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
        delta = librosa.feature.delta(mfcc)

        def pad(x): return np.pad(x, ((0, 0), (0, max(0, 174 - x.shape[1]))), mode='constant')[:, :174]
        features = np.vstack([pad(mfcc), pad(chroma), pad(contrast), pad(delta)])
        features = features[np.newaxis, ..., np.newaxis]  # (1, 99, 174, 1)

        # Step 2: Predict
        preds = model.predict(features)
        result = {
            "Confidence": le_conf.inverse_transform([np.argmax(preds[0])])[0],
            "Energy":     le_energy.inverse_transform([np.argmax(preds[1])])[0],
            "Clarity":    le_clarity.inverse_transform([np.argmax(preds[2])])[0],
            "Persona":    le_persona.inverse_transform([np.argmax(preds[3])])[0],
        }
        return result

    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# ✅ Provide your .wav file path (update as needed)
file_path = "/content/drive/MyDrive/datasets/recording.wav"  # Make sure this file exists

# ✅ Run prediction
result = predict_persona_from_local_file(
    file_path=file_path,
    model=model,
    le_conf=le_conf,
    le_energy=le_energy,
    le_clarity=le_clarity,
    le_persona=le_persona
)

# ✅ Show result
print("\n🎯 Predicted Traits:")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step

🎯 Predicted Traits:
{'Confidence': np.str_('Low'), 'Energy': np.str_('Low'), 'Clarity': np.str_('Clear'), 'Persona': np.str_('Energetic Doer')}


In [ ]:
from google.colab import files
files.download("persona_model_final.h5")
files.download("encoders.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>